# Time Series: Data Acquisition

____

## Exercises: 
### 1. Using the code from the lesson as a guide and the REST API from https://python.zgulde.net/api/v1/items as we did in the lesson, create a dataframe named items that has all of the data for items.

In [1]:
import requests

response_items = requests.get('https://python.zgulde.net/api/v1/items')
response_items

<Response [200]>

In [2]:
# turn that .json content into a dictionary
data_items = response_items.json()

In [3]:
# create a dataframe containing the dictionary created from the .json sent by the api
import pandas as pd

items = pd.DataFrame(data_items['payload']['items'])
items.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar...,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixe...,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice ...,2.61,759283100036,759283100036


### 2. Do the same thing, but for stores (https://python.zgulde.net/api/v1/stores)

In [4]:
response_stores = requests.get('https://python.zgulde.net/api/v1/stores')
response_stores

<Response [200]>

In [5]:
# turn that .json content into a dictionary
data_stores = response_stores.json()

In [6]:
# create a dataframe containing the dictionary created from the .json sent by the api
import pandas as pd

stores = pd.DataFrame(data_stores['payload']['stores'])
stores.head()

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253
1,9255 FM 471 West,San Antonio,2,TX,78251
2,2118 Fredericksburg Rdj,San Antonio,3,TX,78201
3,516 S Flores St,San Antonio,4,TX,78204
4,1520 Austin Hwy,San Antonio,5,TX,78218


### 3. Extract the data for sales (https://python.zgulde.net/api/v1/sales). There are a lot of pages of data here, so your code will need to be a little more complex. Your code should continue fetching data from the next page until all of the data is extracted.

In [7]:
# iterating thru every page and concatenating the sales info from each page, we create a loop

# acquire .json from url
response_sales = requests.get('https://python.zgulde.net/api/v1/sales')

# turn .json content into dictionary
data_sales = response_sales.json()

#turn dictionary into a dataframe
df_sales = pd.DataFrame(data_sales['payload']['sales'])

#Get ready to iterate through all pages 
num_pages = data_sales['payload']['max_page']

# loop through the iterations
for i in range(1,num_pages):

    response_sales = requests.get('https://python.zgulde.net/api/v1/sales')
    data_sales = response_sales.json()
    df_sales = pd.concat([df_sales, pd.DataFrame(data_sales['payload']['sales'])])

In [9]:
df_sales.shape

(915000, 5)

### 4. Save the data in your files to local csv files so that it will be faster to access in the future.

In [ ]:
df_sales.to_csv('sales.csv')

### 5. Combine the data from your three separate dataframes into one large dataframe.

In [10]:
# to merge all data, I look at all 3 dataframes and look for key to merge on

items.head(1)

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013


In [11]:
stores.head(1)

,store_address,store_city,store_id,store_state,store_zipcode
0,12125 Alamo Ranch Pkwy,San Antonio,1,TX,78253


In [12]:
df_sales.head(1)

,item,sale_amount,sale_date,sale_id,store
0,1,13.0,"Tue, 01 Jan 2013 00:00:00 GMT",1,1


In [14]:
df_sales_and_stores = pd.merge(df_sales, stores, how='left', left_on='store' , right_on='store_id')

In [15]:
df_sales_and_stores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915000 entries, 0 to 914999
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   item           915000 non-null  int64  
 1   sale_amount    915000 non-null  float64
 2   sale_date      915000 non-null  object 
 3   sale_id        915000 non-null  int64  
 4   store          915000 non-null  int64  
 5   store_address  915000 non-null  object 
 6   store_city     915000 non-null  object 
 7   store_id       915000 non-null  int64  
 8   store_state    915000 non-null  object 
 9   store_zipcode  915000 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 76.8+ MB


In [16]:
df_all = pd.merge(df_sales_and_stores, items, how='left', left_on='item', right_on='item_id')

In [17]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 915000 entries, 0 to 914999
Data columns (total 16 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   item           915000 non-null  int64  
 1   sale_amount    915000 non-null  float64
 2   sale_date      915000 non-null  object 
 3   sale_id        915000 non-null  int64  
 4   store          915000 non-null  int64  
 5   store_address  915000 non-null  object 
 6   store_city     915000 non-null  object 
 7   store_id       915000 non-null  int64  
 8   store_state    915000 non-null  object 
 9   store_zipcode  915000 non-null  object 
 10  item_brand     915000 non-null  object 
 11  item_id        915000 non-null  int64  
 12  item_name      915000 non-null  object 
 13  item_price     915000 non-null  float64
 14  item_upc12     915000 non-null  object 
 15  item_upc14     915000 non-null  object 
dtypes: float64(2), int64(5), object(9)
memory usage: 118.7+ MB


### 6. Acquire the Open Power Systems Data for Germany, which has been rapidly expanding its renewable energy production in recent years. The data set includes country-wide totals of electricity consumption, wind power production, and solar power production for 2006-2017. You can get the data here: https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv

In [19]:
ops_germany = pd.read_csv('https://raw.githubusercontent.com/jenfly/opsd/master/opsd_germany_daily.csv')
ops_germany

,Date,Consumption,Wind,Solar,Wind+Solar
0,2006-01-01,1069.18400,NaN,NaN,NaN
1,2006-01-02,1380.52100,NaN,NaN,NaN
2,2006-01-03,1442.53300,NaN,NaN,NaN
3,2006-01-04,1457.21700,NaN,NaN,NaN
4,2006-01-05,1477.13100,NaN,NaN,NaN
...,...,...,...,...,...
4378,2017-12-27,1263.94091,394.507,16.530,411.037
4379,2017-12-28,1299.86398,506.424,14.162,520.586
4380,2017-12-29,1295.08753,584.277,29.854,614.131
4381,2017-12-30,1215.44897,721.247,7.467,728.714


### 7. Make sure all the work that you have done above is reproducible. That is, you should put the code above into separate functions in the acquire.py file and be able to re-run the functions and get the same data.

 All created functions are in acquire.py file, which is also in this repo.